## 포스텍 빅데이터 - 데이터 조작 실습
- 최근 3년간 CCTV 증가율과 한국인, 외국인, 고령자의 비율에 따른 데이터 변화
- 구조 파악하기, 서울시 인구 데이터 파악, cctv 데이터와 인구데이터 합치고 분석

In [3]:
ppl = pd.read_csv('dataHandling-main/data/서울인구.csv')
cctv = pd.read_csv('dataHandling-main/data/서울시cctv.csv')
display(ppl.head())
display(cctv.head())

,기간,자치구,세대,인구,인구.1,인구.2,인구.3,인구.4,인구.5,인구.6,인구.7,인구.8,세대당인구,65세이상고령자
0,기간,자치구,세대,합계,합계,합계,한국인,한국인,한국인,등록외국인,등록외국인,등록외국인,세대당인구,65세이상고령자
1,기간,자치구,세대,계,남자,여자,계,남자,여자,계,남자,여자,세대당인구,65세이상고령자
2,2020.3/4,합계,"4,405,833","9,953,009","4,840,912","5,112,097","9,699,232","4,719,170","4,980,062","253,777","121,742","132,035",2.2,"1,552,356"
3,2020.3/4,종로구,"74,861","159,842","77,391","82,451","149,952","73,024","76,928","9,890","4,367","5,523",2,"28,396"
4,2020.3/4,중구,"63,594","135,321","66,193","69,128","125,800","61,526","64,274","9,521","4,667","4,854",1.98,"24,265"


,기관명,소계,2011년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년
0,강 남 구,"5,221","1,944",195.0,316.0,430,546.0,765.0,577.0,448.0
1,강 동 구,"1,879",303,387.0,134.0,59,144.0,194.0,273.0,385.0
2,강 북 구,"1,265",243,88.0,141.0,74,145.0,254.0,1.0,319.0
3,강 서 구,"1,617",219,155.0,118.0,230,187.0,190.0,264.0,254.0
4,관 악 구,"3,985",430,56.0,419.0,487,609.0,619.0,694.0,671.0


In [4]:
# 자치구, 한국인합계, 외국인합계, 고령자 데이터 추출
ppl = ppl[['자치구', '인구', '인구.3', '인구.6', '65세이상고령자']]

# ppl (인구통계)의 index 0, 1 행은 중복이므로 삭제 / 2행의 합계도 불필요하므로 삭제
ppl_ = ppl.drop([0,1,2], axis=0, inplace=False)
ppl_.reset_index(drop=True, inplace=True)

In [5]:
display(ppl_.info())
display(ppl_.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   자치구       25 non-null     object
 1   인구        25 non-null     object
 2   인구.3      25 non-null     object
 3   인구.6      25 non-null     object
 4   65세이상고령자  25 non-null     object
dtypes: object(5)
memory usage: 1.1+ KB


None

,자치구,인구,인구.3,인구.6,65세이상고령자
0,종로구,"159,842","149,952","9,890","28,396"
1,중구,"135,321","125,800","9,521","24,265"
2,용산구,"244,953","229,786","15,167","39,995"
3,성동구,"302,695","295,591","7,104","45,372"
4,광진구,"361,923","348,064","13,859","50,047"


**Dtype이 모두 object로 되어 있다.**
- 모두 int64로 바꿔준다.

In [6]:
# 기존 자료의 숫자가 콤마를 포함한 string이었기 때문에, 먼저 콤마를 없애준다.
ppl_['인구'] = ppl_['인구'].str.replace(',', '')
ppl_['인구.3'] = ppl_['인구.3'].str.replace(',', '')
ppl_['인구.6'] = ppl_['인구.6'].str.replace(',', '')
ppl_['65세이상고령자'] = ppl_['65세이상고령자'].str.replace(',', '')

In [7]:
# 그리고 전체 데이터타입을 int64로 바꾼다. (ignore로 설정해서 '자치구'칼럼 무시)
ppl_ = ppl_.astype({'인구': int, '인구.3': int,
                    '인구.6': int, '65세이상고령자': int})

In [8]:
# 이제 제대로 정수형으로 나온다.
display(ppl_.info())
display(ppl_.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   자치구       25 non-null     object
 1   인구        25 non-null     int64 
 2   인구.3      25 non-null     int64 
 3   인구.6      25 non-null     int64 
 4   65세이상고령자  25 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 1.1+ KB


None

,자치구,인구,인구.3,인구.6,65세이상고령자
0,종로구,159842,149952,9890,28396
1,중구,135321,125800,9521,24265
2,용산구,244953,229786,15167,39995
3,성동구,302695,295591,7104,45372
4,광진구,361923,348064,13859,50047


In [9]:
# 추가적으로 칼럼 명도 rename 해준다
ppl_.rename(columns={ppl.columns[0]: '구별',
                     ppl.columns[1]: '인구수',
                     ppl.columns[2]: '한국인',
                     ppl.columns[3]: '외국인',
                     ppl.columns[4]: '고령자'},
            inplace=True)

In [10]:
# 구별 '외국인비율'과 '고령자비율'을 계산한다
ppl_['외국인비율'] = ppl_['외국인'] / ppl_['인구수'] * 100
ppl_['고령자비율'] = ppl_['고령자'] / ppl_['인구수'] * 100
ppl_.head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,종로구,159842,149952,9890,28396,6.187360,17.765043
1,중구,135321,125800,9521,24265,7.035863,17.931437
2,용산구,244953,229786,15167,39995,6.191800,16.327622
3,성동구,302695,295591,7104,45372,2.346917,14.989346
4,광진구,361923,348064,13859,50047,3.829268,13.828079


In [11]:
# 외국인 비율과 고령자 비율이 각각 가장 높은 곳은 어딜까?
ppl_.sort_values(by='외국인비율', ascending=False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
18,영등포구,405982,375630,30352,60826,7.476194,14.982438
1,중구,135321,125800,9521,24265,7.035863,17.931437
17,금천구,249641,232157,17484,39361,7.003657,15.767041
16,구로구,435751,405579,30172,69312,6.924138,15.906332
2,용산구,244953,229786,15167,39995,6.191800,16.327622


In [12]:
ppl_.sort_values(by='고령자비율', ascending=False).head()

,구별,인구수,한국인,외국인,고령자,외국인비율,고령자비율
8,강북구,313550,309996,3554,62951,1.133472,20.076862
9,도봉구,329300,327248,2052,61388,0.623140,18.641968
1,중구,135321,125800,9521,24265,7.035863,17.931437
0,종로구,159842,149952,9890,28396,6.187360,17.765043
11,은평구,485842,481546,4296,84353,0.884238,17.362229


- 외국인비율은 영등포구가 7.4%로 제일 높고, 그 다음이 중구, 금천구 순이다.
- 고령자비율은 강북구가 무려 20%로 제일 높고, 그 다음이 도봉구, 중구 순이다.

- **중구는 외국인도 많고 고령자도 많은 동네다**

In [13]:
# cctv 데이터를 파악한다.
display(cctv.info())
display(cctv.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   기관명       25 non-null     object 
 1   소계        25 non-null     object 
 2   2011년 이전  23 non-null     object 
 3   2012년     23 non-null     float64
 4   2013년     23 non-null     float64
 5   2014년     25 non-null     object 
 6   2015년     25 non-null     float64
 7   2016년     25 non-null     float64
 8   2017년     25 non-null     float64
 9   2018년     25 non-null     float64
dtypes: float64(6), object(4)
memory usage: 2.3+ KB


None

,기관명,소계,2011년 이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년
0,강 남 구,"5,221","1,944",195.0,316.0,430,546.0,765.0,577.0,448.0
1,강 동 구,"1,879",303,387.0,134.0,59,144.0,194.0,273.0,385.0
2,강 북 구,"1,265",243,88.0,141.0,74,145.0,254.0,1.0,319.0
3,강 서 구,"1,617",219,155.0,118.0,230,187.0,190.0,264.0,254.0
4,관 악 구,"3,985",430,56.0,419.0,487,609.0,619.0,694.0,671.0


- **dtype**도 제각각인 경우가 있고, 굳이 'cctv 개수'니까 float 쓸 필요 없고 int 쓰면 된다.
- **기관명**의 데이터레코드가 '인구통계'의 '구별' 칼럼과 맞으려면 string 사이사이에 있는 space를 지워야 한다.
- **2011년 이전, 2012년, 2013년** 칼럼에 널값도 있는 것 같다.

In [14]:
# 먼저 널값을 지운다
cctv_ = cctv.dropna()
cctv_.isnull().sum()

기관명         0
소계          0
2011년 이전    0
2012년       0
2013년       0
2014년       0
2015년       0
2016년       0
2017년       0
2018년       0
dtype: int64

In [15]:
# index를 다시 배열한다
cctv_.reset_index(drop=True, inplace=True)

In [16]:
# column rename을 한다
cctv_ = cctv_.rename(columns={cctv_.columns[0]: '구별',
                              cctv_.columns[1]: 'cctv수',
                              cctv_.columns[2]: '2011년_이전'})

In [17]:
# dtype을 바꾸기 위해 소계, 2011년_이전, 2014년 데이터의 콤마를 지운다
cctv_['cctv수'] = cctv_['cctv수'].str.replace(',', '')
cctv_['2011년_이전'] = cctv_['2011년_이전'].str.replace(',', '')
cctv_['2014년'] = cctv_['2014년'].str.replace(',', '')

In [18]:
# '구별'만 빼고 모든 칼럼을 int형으로 바꿔줬다.
cctv_.loc[:, cctv_.columns != '구별'] = cctv_.loc[:, cctv_.columns != '구별'].astype(int)

In [19]:
cctv_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   구별        23 non-null     object
 1   cctv수     23 non-null     int64 
 2   2011년_이전  23 non-null     int64 
 3   2012년     23 non-null     int64 
 4   2013년     23 non-null     int64 
 5   2014년     23 non-null     int64 
 6   2015년     23 non-null     int64 
 7   2016년     23 non-null     int64 
 8   2017년     23 non-null     int64 
 9   2018년     23 non-null     int64 
dtypes: int64(9), object(1)
memory usage: 1.9+ KB


In [20]:
# '구별' 칼럼 레코드들의 string 사이 space들을 지운다.
cctv_['구별'] = cctv_['구별'].str.replace(' ', '')

In [21]:
cctv_.head()

,구별,cctv수,2011년_이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년
0,강남구,5221,1944,195,316,430,546,765,577,448
1,강동구,1879,303,387,134,59,144,194,273,385
2,강북구,1265,243,88,141,74,145,254,1,319
3,강서구,1617,219,155,118,230,187,190,264,254
4,관악구,3985,430,56,419,487,609,619,694,671


In [22]:
# cctv 수를 기준으로 오름차순/내림차순 정렬해본다
cctv_.sort_values(by='cctv수', ascending=False).head(10)

,구별,cctv수,2011년_이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년
0,강남구,5221,1944,195,316,430,546,765,577,448
4,관악구,3985,430,56,419,487,609,619,694,671
6,구로구,3227,852,219,349,187,268,326,540,486
15,성북구,3003,779,84,304,241,279,388,285,643
20,은평구,2962,1365,83,99,343,180,296,229,367
13,서초구,2835,1172,91,228,134,215,352,247,396
17,양천구,2775,772,161,185,169,172,349,137,830
14,성동구,2679,665,109,118,101,258,201,933,294
18,영등포구,2495,132,121,206,217,366,289,371,793
12,서대문구,2121,565,233,214,114,109,277,415,194


In [23]:
cctv_.sort_values(by='cctv수', ascending=True).head(10)

,구별,cctv수,2011년_이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년
9,도봉구,858,197,66,8,185,59,155,117,71
2,강북구,1265,243,88,141,74,145,254,1,319
21,종로구,1471,8,7,599,132,195,148,281,101
22,중구,1544,25,165,114,80,245,270,317,328
5,광진구,1581,470,42,83,87,64,21,468,346
16,송파구,1586,600,99,88,21,166,100,116,396
3,강서구,1617,219,155,118,230,187,190,264,254
7,금천구,1634,27,17,242,101,382,136,199,530
10,동작구,1780,238,93,29,503,130,254,278,255
1,강동구,1879,303,387,134,59,144,194,273,385


- cctv가 가장 많은 곳은 강남구, 가장 적은 곳은 도봉구다.

In [24]:
# 최근 3년간 cctv 증가율을 비교해보자
    # "증가율 = 2016~2018 cctv 수 / 2016년 이전 cctv 수"  로 계산한다.
cctv_['최근증가율'] = (cctv_['2018년']+cctv_['2017년']+cctv_['2016년']) / (cctv_['cctv수'] - (cctv_['2018년']+cctv_['2017년']+cctv_['2016년'])) * 100

cctv_.sort_values(by='최근증가율', ascending=False).head()

,구별,cctv수,2011년_이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,최근증가율
22,중구,1544,25,165,114,80,245,270,317,328,145.468998
18,영등포구,2495,132,121,206,217,366,289,371,793,139.443378
14,성동구,2679,665,109,118,101,258,201,933,294,114.148681
7,금천구,1634,27,17,242,101,382,136,199,530,112.483745
5,광진구,1581,470,42,83,87,64,21,468,346,111.930295


- 최근 3년간 cctv가 가장 많이 증가한 곳은 중구, 영등포구, 성동구 순이다.

In [25]:
# 이제 cctv_와 ppl_을 합쳐서 분석해보자
merg_ = pd.merge(cctv_, ppl_, on='구별')
merg_.head()

,구별,cctv수,2011년_이전,2012년,2013년,2014년,2015년,2016년,2017년,2018년,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,5221,1944,195,316,430,546,765,577,448,52.171379,544085,539235,4850,73942,0.891405,13.590156
1,강동구,1879,303,387,134,59,144,194,273,385,82.960078,462275,458058,4217,68798,0.912228,14.882483
2,강북구,1265,243,88,141,74,145,254,1,319,83.068017,313550,309996,3554,62951,1.133472,20.076862
3,강서구,1617,219,155,118,230,187,190,264,254,77.887789,589536,583544,5992,88052,1.016393,14.935814
4,관악구,3985,430,56,419,487,609,619,694,671,99.150425,514555,498574,15981,78206,3.105790,15.198764


In [26]:
# cctv수와 최근증가율을 제외한 2011년_이전 ~ 2018년 열은 지운다
merg_.drop(['2011년_이전', '2012년', '2013년', '2014년', '2015년',
            '2016년', '2017년', '2018년'], axis=1, inplace=True)

In [27]:
merg_.head()

,구별,cctv수,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
0,강남구,5221,52.171379,544085,539235,4850,73942,0.891405,13.590156
1,강동구,1879,82.960078,462275,458058,4217,68798,0.912228,14.882483
2,강북구,1265,83.068017,313550,309996,3554,62951,1.133472,20.076862
3,강서구,1617,77.887789,589536,583544,5992,88052,1.016393,14.935814
4,관악구,3985,99.150425,514555,498574,15981,78206,3.105790,15.198764


- 인구수를 20만이하, 20만, 30만, 40만, 50만이상 으로 카테고리화 해서 비교해보자

In [45]:
merg_['ppl_size'] = 0
merg_.loc[merg_['인구수'] < 200000, 'ppl_size'] = '20만이하'
merg_.loc[(merg_['인구수'] >= 200000) & (merg_['인구수'] < 300000), 'ppl_size'] = '20만'
merg_.loc[(merg_['인구수'] >= 300000) & (merg_['인구수'] < 400000), 'ppl_size'] = '30만'
merg_.loc[(merg_['인구수'] >= 400000) & (merg_['인구수'] < 500000), 'ppl_size'] = '40만'
merg_.loc[merg_['인구수'] >= 500000, 'ppl_size'] = '50만이상'

In [46]:
merg_.head()

,구별,cctv수,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율,ppl_size
0,강남구,5221,52.171379,544085,539235,4850,73942,0.891405,13.590156,50만이상
1,강동구,1879,82.960078,462275,458058,4217,68798,0.912228,14.882483,40만
2,강북구,1265,83.068017,313550,309996,3554,62951,1.133472,20.076862,30만
3,강서구,1617,77.887789,589536,583544,5992,88052,1.016393,14.935814,50만이상
4,관악구,3985,99.150425,514555,498574,15981,78206,3.105790,15.198764,50만이상


In [47]:
merg_.groupby('ppl_size').mean()

,cctv수,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율
ppl_size,,,,,,,,
20만,1848.500000,62.271204,247297.000000,230971.500,16325.500000,39678.000000,6.597729,16.047332
20만이하,1507.500000,100.896029,147581.500000,137876.000,9705.500000,26330.500000,6.611611,17.848240
30만,1739.833333,89.877310,335803.666667,327936.500,7867.166667,54613.333333,2.304098,16.377425
40만,2619.500000,79.892481,441797.625000,429541.875,12255.750000,68204.000000,2.867795,15.418646
50만이상,2863.000000,66.814019,570876.200000,563434.000,7442.200000,83392.200000,1.342978,14.664265


- 눈에 띄는 수치로는..
    - 인구수가 20만이하인 '종로구'와 '중구'에서 cctv의 최근 증가율이 가장 높았다.
    - 아래 데이터를 보면 종로구는 56.3, 중구는 145.5이므로, 중구 때문에 생긴 편향된 값인 것 같다.

In [49]:
merg_[merg_['ppl_size']=='20만이하']

,구별,cctv수,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율,ppl_size
21,종로구,1471,56.323061,159842,149952,9890,28396,6.187360,17.765043,20만이하
22,중구,1544,145.468998,135321,125800,9521,24265,7.035863,17.931437,20만이하


- 또 눈에 띄는 것은 고령자비율과 외국인비율이 '20만'과 '20만이하'의 지역에서 가장 높았다는 것이다.

In [51]:
merg_[(merg_['ppl_size']=='20만이하') | (merg_['ppl_size']=='20만')]

,구별,cctv수,최근증가율,인구수,한국인,외국인,고령자,외국인비율,고령자비율,ppl_size
7,금천구,1634,112.483745,249641,232157,17484,39361,7.003657,15.767041,20만
19,용산구,2063,12.058664,244953,229786,15167,39995,6.191800,16.327622,20만
21,종로구,1471,56.323061,159842,149952,9890,28396,6.187360,17.765043,20만이하
22,중구,1544,145.468998,135321,125800,9521,24265,7.035863,17.931437,20만이하


- 고령자비율의 경우 인구규모가 20만이하 ~ 50만이상 까지 전부 14%~17% 수준에서 움직이기 때문에 큰 의미가 없을 수도 있다.
- 하지만 외국인비율의 경우 30만, 40만, 50만이상 지역이 각각 2.3%, 2.8%, 1.3%로  금천구,용산구,종로구,중구의 6~7%에 비해 현저히 낮다.
    - 비율만 낮은 게 아니라 절대적인 외국인 숫자도 낮다.